In [ ]:
using Revise
using HouseElevation

using CSV
using DataFrames
using DataFramesMeta
using Distributions
using LaTeXStrings
using Metaheuristics
using Plots
using Random
using Unitful

Plots.default(; margin=5Plots.mm)

# Lab 06 Background

In lab-05, we demonstrated that a given decision performs differently under a range of plausible scenarios.  Consider how raising your house 10 feet will prevent a relatively high level of damages if the storm surge is high, but it might cost you more than it saves if future storm surge is low.

However, this complicates analysis when scientists are uncertain what the future scenarios might look like.  In this way, it is necessary to *optimize over an uncertain future*. The best choice can be found by considering a range of possible future scenarios and choosing the decision that performs best on average.

## Decision Variable

Given that this lab optimizes over a range of uncertian futures, our decision variable is the elevation of the house itself. As mentioned in the lab assignment, this problem is simple enough that we can "brute force" serch over our possible elevations (e.g. 0 to 14 feet) to find the best solution.  

## State(s) of the World

In addition to the decision varible, we need to test over many different states of the world (e.g. different sea level rise outcomes, different storm surge values, and different discount rates).  The code below sets up slr_scenarios (the different plausible sea level rise scenarios), draw_surge_distribution (a function that generates a random surge GEV), draw_discount_rate (a function which draws a discount rate from a normal distribution), and the parameters of the house itself.

At the end, the code draws N_SOW state of the worlds and saves them as sows.


In [ ]:
#| output: false
slr_scenarios = let
    df = CSV.read("data/slr_oddo.csv", DataFrame)
    [Oddo17SLR(a, b, c, tstar, cstar) for (a, b, c, tstar, cstar) in eachrow(df)]
end

house = let
    haz_fl_dept = CSV.read("data/haz_fl_dept.csv", DataFrame) # read in the file
    desc = "two story, no basement, Structure"
    occ = "RES1"
    src = "USACE - Galveston"
    row = @rsubset(haz_fl_dept, :Description == desc, :Occupancy == occ, :Source == src)[1, :] 
    area = 881u"ft^2"
    height_above_gauge = 4*u"ft"
    House(row; area=area, height_above_gauge=height_above_gauge, value_usd=194_456)
end

p = ModelParams(; house=house, years=2024:2083)

function draw_surge_distribution()
    μ = rand(Normal(5, 1))
    σ = rand(Exponential(1.5))
    ξ = rand(Normal(0.1, 0.05))
    return GeneralizedExtremeValue(μ, σ, ξ)
end

function draw_discount_rate()
    draw = rand(Normal(0.04, 0.02))
    if draw < 0
        return 0.001
    else
        return draw
    end
end

N_SOW = 100
sows = [
    SOW(rand(slr_scenarios), draw_surge_distribution(), draw_discount_rate()) for
    _ in 1:N_SOW
] 

## Trapezoidal Integration vs. Monte Carlo

We've been estimating yearly expected losses using a Monte Carlo method, which both takes many samples and is not as efficient. Using trapezoidal integration is a better methodology in that it is more accurate and takes less samples.

Below, this new method is tested against the old Monte Carlo methodology.  The code below shows that the new run_sim can finish with around 150 evaluations in a year, while run_sim_old takes around 10,000.


In [ ]:
a = Action(3.0u"ft")
sow = first(sows)
_ = run_sim(a, sow, p)
@time run_sim(a, sow, p)

In [ ]:
_ = HouseElevation.run_sim_old(a, sow, p)
@time HouseElevation.run_sim_old(a, sow, p)

## Metaheuristics.jl

Next, the lab instructions discussed the use of a package, Metaheuristics.jl. This package is frequently used to solve optimization problems, including different optimization algorithms (including genetic algorithms). It gives an example of a function to optimize.

$$
f(\mathbf{x}) = 10D + \sum_{i=1}^{D} \left( x_i^2 - 10 \cos(2 \pi x_i) \right)
$$
where $\mathbf{x} \in [-5, 5]^D$, i.e., $-5 \leq x_i \leq 5$ for $i = 1, \ldots, D$.

The code below shows, visually, how the function values can be minimized.


In [ ]:
f(x) = 10length(x) + sum(x .^ 2 - 10cos.(2π * x))
let
    x = range(-5; stop=5, length=1000)
    y = range(-5; stop=5, length=1000)
    z = [f([i, j]) for i in x, j in y]
    surface(
        x, y, z; xlabel="x1", ylabel="x2", zlabel="f(x)", title=L"Minimize $f(x)$ for $D=2$"
    )
end

This section also demonstrates other key features of the package.  First, **boxconstraints** is used to define the bounds of the decision variable, so we know that when we do our optimization later we use 0 to 14.  It also shows that Metaheuristics.jl is called with **optimize(funtion,bounds)**.  Furthermore, **minimizer** can be used to find which decision variable produced the minimum.  Additional options include adding a time limit and choosing an algorithm.

In addition, a random seed sets up the "random" path taken through optimization, so if we set a specific seed we can get the same result every time we run the optimization.  This is useful for reproducibility.


In [ ]:
D = 10
bounds = boxconstraints(; lb=-5ones(D), ub=5ones(D))
result = optimize(f, bounds)
minimizer(result)
options = Options(; time_limit=10.0)
algorithm = ECA(; options=options)
Random.seed!(918)
result = optimize(f, bounds, algorithm)

# Elevation Optimization

We know that we want to optimize the elevation of the house over a range of different states of the world.  However, we need a specific function to optimize.  This functions tells us the utility of our choice (e.g. it is a function that tells us how good we are doing).  In this case, we set up an **objective_function** that takes in our choice and calculates (1) elevation construction costs and (2) predicted damages over our time period to determine the overall costs.  In this way, it tells us how much money we lose and how good our decision performed.

First, though, we have to set up the states of the world on which the objective function is optimized:

1. We set the random seed, as discussed above, so the "random path" is the same every time we run the optimization.  This gives us the same result every time we run the optimization.

2. 100,000 states of the world are generated and saved as the variable sows.  

3. This line takes this variable and pulls the first N_SOW_opt scenarios.


In [ ]:
# output: false
Random.seed!(2024) ; #<1>
N_SOW = 100_000 ; #<2>
sows = [
    SOW(rand(slr_scenarios), draw_surge_distribution(), draw_discount_rate()) for _ in 1:N_SOW
    ] ;
N_SOW_opt = 500 ; #<3>
sows_opt = sows[1:N_SOW_opt] ;

Here we set up the objective function.

1. The input is converted into an action (e.g. the elevation of the house with units of feet).

2. Run_sim, our simulation with the faster trapezoidal methodology, is run over all states of the world for the action, a. The results are saved as expected_values.  Remembe that this is *the cost of elevating and the expected flood damages combined*.  We want to minimize this cost.

3. We return the sum of the expected values.  Think of this as the cost of damages in every state of the world added together.  It measures how well the action performed across the board.


In [ ]:
function objective_function(a::Vector{Float64})
    action = Action(a[1]*u"ft") #<1>

    expected_values = [run_sim(action, sow, p) for sow in sows_opt] #<2>

    return -sum(expected_values) #<3>
end

Below, the optimization can now be run on the function, objective_function.  We can optimize our decision variable, the action of elevation from 0 to 14 feet, across the different states of the world, using the objective function as our measure of how well we are doing.  The result is the elevation that minimizes the cost of damages and construction.  

1. Here, we set limits on the decision variable, knowing we can only elevate between 0 and 14 feet.

2. We set a time limit of 60 seconds for the optimization.  If it reaches 60 seconds, the optimization will terminate even if it has not converged.

3. We set the algorithm to ECA, which is a type of genetic algorithm.  This is the algorithm that will be used to optimize the decision variable.

4. The optimization is run and the result is saved as result.


In [ ]:
bounds = boxconstraints(; lb=0.0, ub=14.0) #<1>
options = Options(; time_limit=60.0) #<2>
algorithm = ECA(; options=options) #<3>
result = optimize(objective_function, bounds, algorithm) #<4>

The code below provides a sanity check for the optimization.  It plots the objective function for a range of elevations (from 0 to 14 ft) using all your SOWs.  If the recommended elevation is the minimum, then we know the optimization worked.  

1. The range of elevations is set from 0 to 14 feet.

2. The objective function is calculated for each elevation in the range.

3. The objective function is plotted.


In [ ]:
elevations = range(0; stop=14, length=100) #<1>
objective_values = [objective_function(e) for e in elevations] #<2>
plot(elevations, objective_values; xlabel="Elevation (ft)", ylabel="Objective function value", label="", title="Objective function for different elevations, 
Number of SOWs = $N_SOW_opt") #<3>

This plot lines up with the minimizing elevation found by the optimizer: 13.239 feet.  It is the elevation that performs best when averaging across different plausible futures.

At first, this value seemed high.  However, after discussing with Dr. Doss-Gollin, it seems that this high elevation is likely a representation of the fact that (1) this house is already valuable, so any percent damage causes high cost, (2) the discount rate, on average, is low, so we care a lot about future damages, and (3) the sea level rise scenarios seem to be high, putting the house at a high level of risk.

In the end, then, it is clear that this house, being both of high value and close proximity to the water, should be elevated to a high level to avoid future damages.  It is worth it to elevate in order to avoid future flooding.

# Conclusion

Conclude your analysis by reflecting on the following questions

1. How are we framing this problem? What are the decision variables, the objective function, and the states of the world over which we optimize?
    Our decision variable is the height at which we elevate the house.  We want to know which elevation height finds the best balance between the cost of raising a home and the future flood damages we might avoid.  In this way, our objective function determines how much a given decision will cost us over a certain future time period.  We want to minimize this cost.  The states of the world are the different scenarios of sea level rise, storm surge, and discount rates that we might encounter in the future.  
2. Digging deeper, we are averaging the objective function computed over a finite number of states of the world. This assumes that they are all drawn from a distribution representing the "true" distribution of states of the world. Is this a good assumption?
    Scientists and researchers, though they can make valuable predictions, can never know for sure what the true distribution of the future state of the world is.  There will be unexpected outcomes, etc.  This model then uses our "best guess" of what the future might look like, but it may not necessarily be representative of the true future.  This is a limitation of our analysis.
3. What's not being considered in this analysis that might be important?
    There are of course many other added complexities that could be added to this model.  Given my background in flood modeling, I notice that this model considers storm surge, but it does not consider any flooding caused by rainfall.  Often, homes close to the coast are particularly vulnerable to *compound flooding* or the combined effects of storm surge and heavy rainfall (both pluvial and fluvial flooding).  This would be an interesting addition to the model.  
    In addition, the model does not consider the potential for future mitigation measures.  As the home I study is in Galveston, it could be impacted by future projects like the Coastal Spine and the Galveston Bay Park Plan.
    Finally, the model does not consider the potential for future land use changes (e.g. continuing development of the island).